# Final model
Here the best combination of predictor and dataset features will be determined and validated.

## Model selection with feature combinations
We evaluate multiple feature sets (baseline, holidays, Fourier, trend, and their combination) together with different predictors. Each predictor is fine-tuned via a small hyperparameter grid, and the best-performing combination on the validation split is returned.

In [4]:
from __future__ import annotations
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from sklearn.metrics import mean_absolute_error, root_mean_squared_error
from sklearn.model_selection import ParameterGrid
from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, AutoETS, SeasonalNaive
from statsforecast.models import AutoARIMA, AutoETS, SeasonalNaive
import inspect

DATA_DIR = Path("..") / "data" / "processed_data"
FEATURE_KEYS = ["none", "holidays", "fourier", "trend", "fourier+trend+holidays"]
SPLITS = ["train", "val"]
FREQ = "D"

def load_feature_sets():
    features = {k: {} for k in FEATURE_KEYS}
    future = {k: {} for k in FEATURE_KEYS}
    for key in FEATURE_KEYS:
        for split in SPLITS:
            path = DATA_DIR / f"{key}_{split}.parquet"
            features[key][split] = pl.read_parquet(path) if path.exists() else None
            future_path = DATA_DIR / f"{key}_{split}_future.parquet"
            future[key][split] = pl.read_parquet(future_path) if future_path.exists() else None
    return features, future

def prepare_target(df: pl.DataFrame) -> pd.DataFrame:
    return df.select(["unique_id", "ds", "y"]).to_pandas()

def prepare_exog(df: pl.DataFrame | None) -> pd.DataFrame | None:
    if df is None:
        return None
    exog_cols = [c for c in df.columns if c not in ("unique_id", "ds", "y")]
    if not exog_cols:
        return None
    exog = df.select(["unique_id", "ds", *exog_cols]).to_pandas()
    # only dummy-encode true exogenous categoricals; keep unique_id/ds intact
    cat_cols = [c for c in exog_cols if exog[c].dtype == "object"]
    if cat_cols:
        exog = pd.get_dummies(exog, columns=cat_cols, drop_first=False)
    return exog

features, features_future = load_feature_sets()
features

{'none': {'train': shape: (19_180, 3)
  ┌────────────┬──────────────┬────────┐
  │ ds         ┆ unique_id    ┆ y      │
  │ ---        ┆ ---          ┆ ---    │
  │ date       ┆ str          ┆ f64    │
  ╞════════════╪══════════════╪════════╡
  │ 2020-07-01 ┆ "Bubble tea" ┆ 2012.0 │
  │ 2020-07-02 ┆ "Bubble tea" ┆ 2085.0 │
  │ 2020-07-03 ┆ "Bubble tea" ┆ 2204.0 │
  │ 2020-07-04 ┆ "Bubble tea" ┆ 2119.0 │
  │ 2020-07-05 ┆ "Bubble tea" ┆ 2176.0 │
  │ …          ┆ …            ┆ …      │
  │ 2025-09-26 ┆ "Tea"        ┆ 1206.0 │
  │ 2025-09-27 ┆ "Tea"        ┆ 1074.0 │
  │ 2025-09-28 ┆ "Tea"        ┆ 1222.0 │
  │ 2025-09-29 ┆ "Tea"        ┆ 1211.0 │
  │ 2025-09-30 ┆ "Tea"        ┆ 1220.0 │
  └────────────┴──────────────┴────────┘,
  'val': shape: (310, 3)
  ┌────────────┬──────────────┬────────┐
  │ ds         ┆ unique_id    ┆ y      │
  │ ---        ┆ ---          ┆ ---    │
  │ date       ┆ str          ┆ f64    │
  ╞════════════╪══════════════╪════════╡
  │ 2025-10-01 ┆ "Bubble tea" ┆ 11

In [20]:
from sklearn.metrics import r2_score

In [ ]:
def get_horizon(df_future: pl.DataFrame | None, val_df: pl.DataFrame) -> int:
    if df_future is not None:
        first_id = df_future["unique_id"][0]
        return df_future.filter(pl.col("unique_id") == first_id).height
    first_id = val_df["unique_id"][0]
    return val_df.filter(pl.col("unique_id") == first_id).height

MODEL_GRID = {
    "AutoARIMA": {
        "constructor": AutoARIMA,
        "params": {
            "season_length": [7],
            "stepwise": [True],
            "approximation": [True, False],
        },
    },
    "AutoETS": {
        "constructor": AutoETS,
        "params": {
            "season_length": [7],
            "model": ["ZZZ"],
        },
    },
    "SeasonalNaive": {
        "constructor": SeasonalNaive,
        "params": {
            "season_length": [7, 14],
        },
    },
}





def _fit_with_exog(sf: StatsForecast, df: pd.DataFrame, X: pd.DataFrame | None):
    sig = inspect.signature(sf.fit)
    kwargs = {}
    if X is not None:
        if "X_df" in sig.parameters:
            kwargs["X_df"] = X
        elif "X" in sig.parameters:
            kwargs["X"] = X
    return sf.fit(df=df, **kwargs)

def _predict_with_exog(fitted: StatsForecast, h: int, X: pd.DataFrame | None):
    sig = inspect.signature(fitted.predict)
    kwargs = {}
    if X is not None:
        if "X_df" in sig.parameters:
            kwargs["X_df"] = X
        elif "X" in sig.parameters:
            kwargs["X"] = X
    return fitted.predict(h=h, **kwargs)

def align_exog(train_exog: pd.DataFrame | None, fut_exog: pd.DataFrame | None):
    if train_exog is None or fut_exog is None:
        return train_exog, fut_exog
    cols = sorted(set(train_exog.columns) | set(fut_exog.columns))
    train_aligned = train_exog.reindex(columns=cols, fill_value=0)
    fut_aligned = fut_exog.reindex(columns=cols, fill_value=0)
    return train_aligned, fut_aligned



def evaluate_feature_model(feature_key: str):
    train = features[feature_key].get("train")
    val = features[feature_key].get("val")
    val_future = features_future[feature_key].get("val")
    if train is None or val is None:
        return []
    y_train = prepare_target(train)
    y_val = prepare_target(val)
    X_train = prepare_exog(train)
    X_val_future = prepare_exog(val_future)
    X_train, X_val_future = align_exog(X_train, X_val_future)
    h = get_horizon(val_future, val)
    


    results = []
    for model_name, cfg in MODEL_GRID.items():
        for params in ParameterGrid(cfg["params"]):
            model = cfg["constructor"](**params)
            sf = StatsForecast(models=[model], freq=FREQ, n_jobs=-1)
            
            # Try with exogenous variables first
            try:
                fitted = _fit_with_exog(sf, y_train, X_train)
                fcst = _predict_with_exog(fitted, h, X_val_future)
                used_exog = True
            except (ValueError, TypeError) as e:
                # If shape mismatch or other exog error, retry without exogenous
                if "shape" in str(e) or "unexpected keyword" in str(e):
                    print(f"  Skipping exog for {feature_key}/{model_name} due to: {e}")
                    sf = StatsForecast(models=[model], freq=FREQ, n_jobs=-1)
                    fitted = sf.fit(df=y_train)
                    fcst = fitted.predict(h=h)
                    used_exog = False
                else:
                    raise
            
            yhat_col = [c for c in fcst.columns if c not in ("unique_id", "ds")][0]
            preds = fcst.rename(columns={yhat_col: "yhat"})
            merged = y_val.merge(preds, on=["unique_id", "ds"], how="inner")
            mae = mean_absolute_error(merged["y"], merged["yhat"])
            rmse = root_mean_squared_error(merged["y"], merged["yhat"])
            smape = np.mean(200 * np.abs(merged["yhat"] - merged["y"]) / (np.abs(merged["y"]) + np.abs(merged["yhat"]) + 1e-8))
            r2 = r2_score(y_val, val_future)
            results.append({
                "feature_set": feature_key,
                "model": model_name,
                "params": params,
                "used_exog": used_exog,
                "mae": mae,
                "rmse": rmse,
                "smape": smape,
                "r2_Score": r2
            })
    return results

In [6]:
all_results: list[dict] = []
for feature_key in FEATURE_KEYS:
    all_results.extend(evaluate_feature_model(feature_key))

if not all_results:
    raise RuntimeError("No feature/model results were produced. Ensure processed parquet files exist.")

results_df = pd.DataFrame(all_results).sort_values(["rmse", "mae"]).reset_index(drop=True)
display(results_df.head(10))

best_combo = results_df.iloc[0]
best_combo

  Skipping exog for holidays/AutoARIMA due to: Expected X to have shape (310, 2), but got (310, 23)
  Skipping exog for holidays/AutoARIMA due to: Expected X to have shape (310, 2), but got (310, 23)
  Skipping exog for holidays/AutoETS due to: Expected X to have shape (310, 2), but got (310, 23)
  Skipping exog for holidays/SeasonalNaive due to: Expected X to have shape (310, 2), but got (310, 23)
  Skipping exog for holidays/SeasonalNaive due to: Expected X to have shape (310, 2), but got (310, 23)
  Skipping exog for fourier/AutoARIMA due to: Expected X to have shape (310, 2), but got (310, 8)
  Skipping exog for fourier/AutoARIMA due to: Expected X to have shape (310, 2), but got (310, 8)
  Skipping exog for fourier/AutoETS due to: Expected X to have shape (310, 2), but got (310, 8)
  Skipping exog for fourier/SeasonalNaive due to: Expected X to have shape (310, 2), but got (310, 8)
  Skipping exog for fourier/SeasonalNaive due to: Expected X to have shape (310, 2), but got (310, 8

,feature_set,model,params,used_exog,mae,rmse,smape
0,none,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",True,78.507702,169.319382,19.019246
1,holidays,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246
2,fourier,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246
3,trend,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246
4,fourier+trend+holidays,AutoETS,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246
5,none,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",True,81.335207,174.744147,19.149107
6,holidays,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107
7,fourier,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107
8,trend,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107
9,fourier+trend+holidays,AutoARIMA,"{'approximation': True, 'season_length': 7, 's...",False,81.335207,174.744147,19.149107


feature_set                                    none
model                                       AutoETS
params         {'model': 'ZZZ', 'season_length': 7}
used_exog                                      True
mae                                       78.507702
rmse                                     169.319382
smape                                     19.019246
Name: 0, dtype: object

## Test set evaluation
Load test split, retrain best model on train+val, and generate predictions against baseline.

In [7]:
# Load test split
SPLITS_TEST = ["train", "val", "test"]

def load_all_splits(feature_key: str):
    splits_data = {}
    futures_data = {}
    for split in SPLITS_TEST:
        path = DATA_DIR / f"{feature_key}_{split}.parquet"
        splits_data[split] = pl.read_parquet(path) if path.exists() else None
        future_path = DATA_DIR / f"{feature_key}_{split}_future.parquet"
        futures_data[split] = pl.read_parquet(future_path) if future_path.exists() else None
    return splits_data, futures_data

best_feature = best_combo["feature_set"]
best_model_name = best_combo["model"]
best_params = best_combo["params"]
used_exog = best_combo["used_exog"]

print(f"Best: {best_model_name} with {best_feature} features")
print(f"Params: {best_params}")
print(f"Used exog: {used_exog}")

splits, futures = load_all_splits(best_feature)
splits.keys()

Best: AutoETS with none features
Params: {'model': 'ZZZ', 'season_length': 7}
Used exog: True


dict_keys(['train', 'val', 'test'])

In [9]:
# Combine train+val for final training
y_train_val = pd.concat([
    prepare_target(splits["train"]),
    prepare_target(splits["val"])
], ignore_index=True).sort_values(["unique_id", "ds"]).reset_index(drop=True)

y_test = prepare_target(splits["test"])

if used_exog:
    exog_train = prepare_exog(splits["train"])
    exog_val = prepare_exog(splits["val"])
    
    if exog_train is not None and exog_val is not None:
        X_train_val = pd.concat([exog_train, exog_val], ignore_index=True).sort_values(["unique_id", "ds"]).reset_index(drop=True)
        X_test_future = prepare_exog(futures["test"])
        X_train_val, X_test_future = align_exog(X_train_val, X_test_future)
    else:
        print("Warning: Exog features missing in train/val, falling back to no exog")
        X_train_val = None
        X_test_future = None
        used_exog = False
else:
    X_train_val = None
    X_test_future = None
h_test = get_horizon(futures["test"], splits["test"])
print(f"Train+Val shape: {y_train_val.shape}")
print(f"Test shape: {y_test.shape}")
print(f"Test horizon: {h_test}")

Train+Val shape: (19490, 3)
Test shape: (310, 3)
Test horizon: 31


In [10]:
# Train baseline (SeasonalNaive 7-day)
baseline_model = SeasonalNaive(season_length=7)
sf_baseline = StatsForecast(models=[baseline_model], freq=FREQ, n_jobs=-1)
sf_baseline.fit(df=y_train_val)
fcst_baseline = sf_baseline.predict(h=h_test)

baseline_col = [c for c in fcst_baseline.columns if c not in ("unique_id", "ds")][0]
fcst_baseline = fcst_baseline.rename(columns={baseline_col: "baseline_pred"})

# Train best model
best_model_constructor = MODEL_GRID[best_model_name]["constructor"]
best_model = best_model_constructor(**best_params)
sf_best = StatsForecast(models=[best_model], freq=FREQ, n_jobs=-1)

try:
    fitted_best = _fit_with_exog(sf_best, y_train_val, X_train_val)
    fcst_best = _predict_with_exog(fitted_best, h_test, X_test_future)
except (ValueError, TypeError) as e:
    if "shape" in str(e) or "unexpected keyword" in str(e):
        print(f"Falling back to no exog due to: {e}")
        sf_best = StatsForecast(models=[best_model], freq=FREQ, n_jobs=-1)
        fitted_best = sf_best.fit(df=y_train_val)
        fcst_best = fitted_best.predict(h=h_test)
    else:
        raise

best_col = [c for c in fcst_best.columns if c not in ("unique_id", "ds")][0]
fcst_best = fcst_best.rename(columns={best_col: "best_pred"})

print("✓ Baseline and best model trained")

✓ Baseline and best model trained


In [11]:
# Merge predictions with actuals
results_test = y_test.merge(fcst_baseline, on=["unique_id", "ds"], how="left")
results_test = results_test.merge(fcst_best, on=["unique_id", "ds"], how="left")

# Calculate metrics
mae_baseline = mean_absolute_error(results_test["y"], results_test["baseline_pred"])
rmse_baseline = root_mean_squared_error(results_test["y"], results_test["baseline_pred"])
smape_baseline = np.mean(200 * np.abs(results_test["baseline_pred"] - results_test["y"]) / (np.abs(results_test["y"]) + np.abs(results_test["baseline_pred"]) + 1e-8))

mae_best = mean_absolute_error(results_test["y"], results_test["best_pred"])
rmse_best = root_mean_squared_error(results_test["y"], results_test["best_pred"])
smape_best = np.mean(200 * np.abs(results_test["best_pred"] - results_test["y"]) / (np.abs(results_test["y"]) + np.abs(results_test["best_pred"]) + 1e-8))

metrics = pd.DataFrame({
    "model": ["Baseline (SeasonalNaive-7)", f"Best ({best_model_name})"],
    "feature_set": ["none", best_feature],
    "mae": [mae_baseline, mae_best],
    "rmse": [rmse_baseline, rmse_best],
    "smape": [smape_baseline, smape_best],
})

display(metrics)
metrics

,model,feature_set,mae,rmse,smape
0,Baseline (SeasonalNaive-7),none,100.566129,164.751355,24.294597
1,Best (AutoETS),none,86.852207,153.463049,17.512230


,model,feature_set,mae,rmse,smape
0,Baseline (SeasonalNaive-7),none,100.566129,164.751355,24.294597
1,Best (AutoETS),none,86.852207,153.463049,17.512230


In [12]:
# Save predictions and metrics
PRED_DIR = Path("..") / "data" / "predictions"
PRED_DIR.mkdir(parents=True, exist_ok=True)

# Save predictions
results_test_pl = pl.from_pandas(results_test)
pred_path = PRED_DIR / "test_predictions.parquet"
results_test_pl.write_parquet(pred_path)
print(f"Saved predictions to {pred_path}")

# Save metrics
metrics_pl = pl.from_pandas(metrics)
metrics_path = PRED_DIR / "test_metrics.parquet"
metrics_pl.write_parquet(metrics_path)
print(f"Saved metrics to {metrics_path}")

# Save best model configuration
best_config = pd.DataFrame([{
    "model": best_model_name,
    "feature_set": best_feature,
    "params": str(best_params),
    "used_exog": used_exog,
    "val_mae": best_combo["mae"],
    "val_rmse": best_combo["rmse"],
    "val_smape": best_combo["smape"],
    "test_mae": mae_best,
    "test_rmse": rmse_best,
    "test_smape": smape_best,
}])

config_pl = pl.from_pandas(best_config)
config_path = PRED_DIR / "best_model_config.parquet"
config_pl.write_parquet(config_path)
print(f"Saved best model config to {config_path}")

Saved predictions to ..\data\predictions\test_predictions.parquet
Saved metrics to ..\data\predictions\test_metrics.parquet
Saved best model config to ..\data\predictions\best_model_config.parquet


In [19]:
example_1 = pd.read_parquet("../data/predictions/best_model_config.parquet")
example_2 = pd.read_parquet("../data/predictions/test_metrics.parquet")
example_3 = pd.read_parquet("../data/predictions/test_predictions.parquet")
example_1

,model,feature_set,params,used_exog,val_mae,val_rmse,val_smape,test_mae,test_rmse,test_smape
0,AutoETS,none,"{'model': 'ZZZ', 'season_length': 7}",False,78.507702,169.319382,19.019246,86.852207,153.463049,17.51223
